<a href="https://colab.research.google.com/github/gagan3012/EAIS-Temporal-Bias/blob/master/RQ2_Get_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread_pandas gspread-formatting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2


In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
from google.colab import auth
from google.auth import default
import gspread
from gspread_formatting import get_effective_format

# Authenticate and set up Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the spreadsheet
spreadsheet = gc.open('Temporal biases of Foundation Models')
sheet = spreadsheet.worksheet('Actual Evals (2)')  # Open the specific worksheet

# Fetch data from the sheet
data = sheet.get_all_values()

# Convert data to a Pandas DataFrame
df = pd.DataFrame(data)

# Set headers and reset index
df.columns = df.iloc[0]
df = df.iloc[1:].reset_index(drop=True)

# Initialize a DataFrame for cell background colors
df_colors = pd.DataFrame(index=df.index, columns=df.columns)

# Get cell formatting (including background colors) with progress tracking
cell_formats = {}
for row_idx, row in tqdm(enumerate(data, start=1), desc="Processing rows", total=len(data)):
    for col_idx, cell_value in enumerate(row, start=1):
        # Get effective format of the cell
        cell_format = get_effective_format(sheet, f"{chr(64 + col_idx)}{row_idx}")
        if cell_format and cell_format.backgroundColor:  # Check if color exists
            bg_color = cell_format.backgroundColor
            # Store RGB values
            cell_formats[(row_idx - 1, col_idx - 1)] = (
                getattr(bg_color, "red", 1),   # Default to 1 if no value
                getattr(bg_color, "green", 1),
                getattr(bg_color, "blue", 1)
            )

# Populate df_colors with the background colors
for (row, col), color in cell_formats.items():
    if row < len(df_colors) and col < len(df_colors.columns):
        df_colors.iloc[row, col] = color

# Merge data from column 6 onward into the DataFrame
additional_columns = df.iloc[:, 5:]  # Columns from index 5 (6th column) onward
additional_columns = additional_columns.reset_index(drop=True)

# Update the main DataFrame
df = df.iloc[:, :5]  # Retain only the first 5 columns
for col in additional_columns.columns:
    df[col] = additional_columns[col]

# Create the final output with data and colors
df_with_colors = {"data": df, "colors": df_colors}

# Display the resulting data and color metadata
print(df_with_colors)


Processing rows:   0%|          | 0/191 [00:00<?, ?it/s]

{'data': 0    Type of Q                                           Question  \
0      Factual  Which from the following famous people died \n...   
1      Factual  Which from the following famous people died \n...   
2      Factual  Which from the following famous people died \n...   
3      Factual  Which from the following famous people died \n...   
4      Factual  Which from the following famous people died \n...   
..         ...                                                ...   
185  Numerical  What is the time 7 year and 9 month after 27/1...   
186  Numerical  What is the time 7 year and 9 month after 2710...   
187  Numerical  What is the time 7 year and 9 month after 1027...   
188  Numerical  What is the time 7 year and 9 month after 10/2...   
189  Numerical  What is the time 7 year and 9 month after Octo...   

0    Extracted Date                                         Answer  \
0          23041616  William Shakespeare \nOR\nMiguel de Cervantes   
1           4231616  W

<ipython-input-2-b5a646a1cc80>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = additional_columns[col]
<ipython-input-2-b5a646a1cc80>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = additional_columns[col]
<ipython-input-2-b5a646a1cc80>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [ ]:
df

,Type of Q,Question,Extracted Date,Answer,Time Period,Date Format,meta-llama/Meta-Llama-3-70B-Instruct,meta-llama/Meta-Llama-3-8B-Instruct,Qwen/Qwen2.5-72B-Instruct,meta-llama/Llama-3.2-3B-Instruct,meta-llama/Llama-2-7b-chat-hf,microsoft/Phi-3.5-mini-instruct,mistralai/Mistral-7B-Instruct-v0.3,GPT-4o-mini,GPT-4o,GPT-3.5,GPT-4-turbo,CohereForAI/c4ai-command-r-plus-08-2024
0,Factual,Which from the following famous people died \n...,23041616,William Shakespeare \nOR\nMiguel de Cervantes,Past,DDMMYYYY,B) Miguel de Cervantes and D) William Shakespe...,B) Miguel de Cervantes,"B, D",C. Princess Diana,C) Princess Diana died on 23/04/1616.,Princess Diana,B) Miguel de Cervantes,D) William Shakespeare,B) Miguel de Cervantes,B) Miguel de Cervantes,D) William Shakespeare,A
1,Factual,Which from the following famous people died \n...,4231616,William Shakespeare \nOR\nMiguel de Cervantes,Past,MMDDYYYY,B) Miguel de Cervantes,"C) Princess Diana (August 31, 1997, not April ...","B, D",B) Miguel de Cervantes,C) Princess Diana,D) William Shakespeare died on 4th March 1616.,B) Miguel de Cervantes,D) William Shakespeare,B) Miguel de Cervantes,B) Miguel de Cervantes,D) William Shakespeare,B) Miguel de Cervantes.
2,Factual,Which from the following famous people died \n...,23April1616,William Shakespeare \nOR\nMiguel de Cervantes,Past,DDMonYYYY,B) Miguel de Cervantes and D) William Shakespe...,B) Miguel de Cervantes,"B, D",B) Miguel de Cervantes,"D) William Shakespeare died on April 23, 1616.",D) William Shakespeare,D) William Shakespeare,B) Miguel de Cervantes,B) Miguel de Cervantes,D) William Shakespeare,D) William Shakespeare,C) Princess Diana
3,Factual,Which from the following famous people died \n...,23-04-16,William Shakespeare \nOR\nMiguel de Cervantes,Past,DD-MM-YY,D) William Shakespeare,None of the above.,"B, D",A) Shah Jahan.,C) Princess Diana,C) Princess Diana,B) Miguel de Cervantes,D) William Shakespeare,"B, D",C) Princess Diana,D) William Shakespeare,D.
4,Factual,Which from the following famous people died \n...,"1616, April 23",William Shakespeare \nOR\nMiguel de Cervantes,Past,"YYYY, Mon DD",B) Miguel de Cervantes and D) William Shakespe...,B) Miguel de Cervantes,"B, D",D) William Shakespeare,C) Princess Diana,B) Miguel de Cervantes,B) Miguel de Cervantes,B) Miguel de Cervantes,B) Miguel de Cervantes,D) William Shakespeare,B) Miguel de Cervantes and D) William Shakespe...,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,Numerical,What is the time 7 year and 9 month after 27/1...,,"Jul 27, 1454",Past,,The time is 3/7/1534.,"July 6, 1555",15/07/1454,"To find the date, add 7 years and 9 months to ...",The time 7 years and 9 months after 27/10/1446...,Calculating the date 7 years and 9 months afte...,"The time would be 6th July, 1555. (Using Islam...",07/07/1454,27/06/1454,09/07/1454,The date would be 27/07/1454.,"I'm sorry, the reference time you gave me is e..."
186,Numerical,What is the time 7 year and 9 month after 2710...,,"Jul 27, 1454",Past,,27101446 + 7 years + 9 months = 27111475,"Add 7 years and 9 months to 2710, resulting in...",27183346,I cannot provide a date in Gregorian format fo...,The time 7 years and 9 months after 27101446 i...,27113356 (27 Jan 2010 + 7 years + 9 months),The time is 27406152 hours after 27101446.\n\n...,The result is 27070753.,"July 20, 2024.",28018134,"Time after 27101446: 28091446 (adding 7 years,...",The time 7 years and 9 months after 27101446 i...
187,Numerical,What is the time 7 year and 9 month after 1027...,,"Jul 27, 1454",Past,,"October 27, 2040.","7 years, 9 months after 10271446 = 10311655.",10271446 + 7 years 9 months = 10353406.,I cannot help with calculations that involve a...,"Time after 10271446: 9 months = Oct 1028, 7 ye...",10360346 (calculated by adding 7 years and 9 m...,"The time would be 11 years, 9 months, and 14 s...",Add 7 years and 9 months to the date 10271446.,October 2023 plus 7 years and 9 months is July...,11039198,Time: 10429846 (Unix timesta

In [ ]:
df_colors

,Type of Q,Question,Extracted Date,Answer,Time Period,Date Format,meta-llama/Meta-Llama-3-70B-Instruct,meta-llama/Meta-Llama-3-8B-Instruct,Qwen/Qwen2.5-72B-Instruct,meta-llama/Llama-3.2-3B-Instruct,meta-llama/Llama-2-7b-chat-hf,microsoft/Phi-3.5-mini-instruct,mistralai/Mistral-7B-Instruct-v0.3,GPT-4o-mini,GPT-4o,GPT-3.5,GPT-4-turbo,CohereForAI/c4ai-command-r-plus-08-2024
0,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)"
1,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)"
2,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.43529412, 0.65882355, 0.8627451)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)"
3,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)"
4,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.9647059, 0.69803923, 0.41960785)"
186,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)"
187,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)"
188,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.9176470

In [ ]:
df.to_csv('data.csv', index=False)
df_colors.to_csv('colors.csv', index=False)

In [ ]:
df_colors

,Type of Q,Question,Extracted Date,Answer,Time Period,Date Format,meta-llama/Meta-Llama-3-70B-Instruct,meta-llama/Meta-Llama-3-8B-Instruct,Qwen/Qwen2.5-72B-Instruct,meta-llama/Llama-3.2-3B-Instruct,meta-llama/Llama-2-7b-chat-hf,microsoft/Phi-3.5-mini-instruct,mistralai/Mistral-7B-Instruct-v0.3,GPT-4o-mini,GPT-4o,GPT-3.5,GPT-4-turbo,CohereForAI/c4ai-command-r-plus-08-2024
0,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)"
1,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)"
2,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.43529412, 0.65882355, 0.8627451)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)"
3,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)"
4,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.9647059, 0.69803923, 0.41960785)"
186,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)"
187,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)"
188,"(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(1, 1, 1)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.9176470

In [74]:
import pandas as pd

df = pd.read_csv('data_eval.csv')
df_colors = pd.read_csv('colors.csv')

In [75]:
df

,Type of Q,Question,Extracted Date,Answer,Time Period,Date Format,meta-llama/Meta-Llama-3-70B-Instruct,meta-llama/Meta-Llama-3-8B-Instruct,Qwen/Qwen2.5-72B-Instruct,meta-llama/Llama-3.2-3B-Instruct,meta-llama/Llama-2-7b-chat-hf,microsoft/Phi-3.5-mini-instruct,mistralai/Mistral-7B-Instruct-v0.3,GPT-4o-mini,GPT-4o,GPT-3.5,GPT-4-turbo,CohereForAI/c4ai-command-r-plus-08-2024
0,Factual,Which from the following famous people died \n...,23041616,William Shakespeare \nOR\nMiguel de Cervantes,Past,DDMMYYYY,B) Miguel de Cervantes and D) William Shakespe...,B) Miguel de Cervantes,"B, D",C. Princess Diana,C) Princess Diana died on 23/04/1616.,Princess Diana,B) Miguel de Cervantes,D) William Shakespeare,B) Miguel de Cervantes,B) Miguel de Cervantes,D) William Shakespeare,A
1,Factual,Which from the following famous people died \n...,4231616,William Shakespeare \nOR\nMiguel de Cervantes,Past,MMDDYYYY,B) Miguel de Cervantes,"C) Princess Diana (August 31, 1997, not April ...","B, D",B) Miguel de Cervantes,C) Princess Diana,D) William Shakespeare died on 4th March 1616.,B) Miguel de Cervantes,D) William Shakespeare,B) Miguel de Cervantes,B) Miguel de Cervantes,D) William Shakespeare,B) Miguel de Cervantes.
2,Factual,Which from the following famous people died \n...,23April1616,William Shakespeare \nOR\nMiguel de Cervantes,Past,DDMonYYYY,B) Miguel de Cervantes and D) William Shakespe...,B) Miguel de Cervantes,"B, D",B) Miguel de Cervantes,"D) William Shakespeare died on April 23, 1616.",D) William Shakespeare,D) William Shakespeare,B) Miguel de Cervantes,B) Miguel de Cervantes,D) William Shakespeare,D) William Shakespeare,C) Princess Diana
3,Factual,Which from the following famous people died \n...,23-04-16,William Shakespeare \nOR\nMiguel de Cervantes,Past,DD-MM-YY,D) William Shakespeare,None of the above.,"B, D",A) Shah Jahan.,C) Princess Diana,C) Princess Diana,B) Miguel de Cervantes,D) William Shakespeare,"B, D",C) Princess Diana,D) William Shakespeare,D.
4,Factual,Which from the following famous people died \n...,"1616, April 23",William Shakespeare \nOR\nMiguel de Cervantes,Past,"YYYY, Mon DD",B) Miguel de Cervantes and D) William Shakespe...,B) Miguel de Cervantes,"B, D",D) William Shakespeare,C) Princess Diana,B) Miguel de Cervantes,B) Miguel de Cervantes,B) Miguel de Cervantes,B) Miguel de Cervantes,D) William Shakespeare,B) Miguel de Cervantes and D) William Shakespe...,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,Numerical,What is the time 7 year and 9 month after 27/1...,NaN,"Jul 27, 1454",Past,NaN,The time is 3/7/1534.,"July 6, 1555",15/07/1454,"To find the date, add 7 years and 9 months to ...",The time 7 years and 9 months after 27/10/1446...,Calculating the date 7 years and 9 months afte...,"The time would be 6th July, 1555. (Using Islam...",07/07/1454,27/06/1454,09/07/1454,The date would be 27/07/1454.,"I'm sorry, the reference time you gave me is e..."
186,Numerical,What is the time 7 year and 9 month after 2710...,NaN,"Jul 27, 1454",Past,NaN,27101446 + 7 years + 9 months = 27111475,"Add 7 years and 9 months to 2710, resulting in...",27183346,I cannot provide a date in Gregorian format fo...,The time 7 years and 9 months after 27101446 i...,27113356 (27 Jan 2010 + 7 years + 9 months),The time is 27406152 hours after 27101446.\n\n...,The result is 27070753.,"July 20, 2024.",28018134,"Time after 27101446: 28091446 (adding 7 years,...",The time 7 years and 9 months after 27101446 i...
187,Numerical,What is the time 7 year and 9 month after 1027...,NaN,"Jul 27, 1454",Past,NaN,"October 27, 2040.","7 years, 9 months after 10271446 = 10311655.",10271446 + 7 years 9 months = 10353406.,I cannot help with calculations that involve a...,"Time after 10271446: 9 months = Oct 1028, 7 ye...",10360346 (calculated by adding 7 years and 9 m...,"The time would be 11 years, 9 months, and 14 s...",Add 7 years and 9 months to the date 10271446.,October 2023 plus 7 years and 9 months is July...,11039198,Time: 1042

In [76]:
#drop first row
df_colors = df_colors.iloc[1:]
df_colors = df_colors.reset_index(drop=True)
#drop the last row of df
df = df.iloc[:-1]
df = df.reset_index(drop=True)

In [77]:
# replace the first 5 columns of df_colors with df
df_colors.iloc[:, :6] = df.iloc[:, :6]

In [78]:
df_colors

,Type of Q,Question,Extracted Date,Answer,Time Period,Date Format,meta-llama/Meta-Llama-3-70B-Instruct,meta-llama/Meta-Llama-3-8B-Instruct,Qwen/Qwen2.5-72B-Instruct,meta-llama/Llama-3.2-3B-Instruct,meta-llama/Llama-2-7b-chat-hf,microsoft/Phi-3.5-mini-instruct,mistralai/Mistral-7B-Instruct-v0.3,GPT-4o-mini,GPT-4o,GPT-3.5,GPT-4-turbo,CohereForAI/c4ai-command-r-plus-08-2024
0,Factual,Which from the following famous people died \n...,23041616,William Shakespeare \nOR\nMiguel de Cervantes,Past,DDMMYYYY,"(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)"
1,Factual,Which from the following famous people died \n...,4231616,William Shakespeare \nOR\nMiguel de Cervantes,Past,MMDDYYYY,"(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.43529412, 0.65882355, 0.8627451)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)"
2,Factual,Which from the following famous people died \n...,23April1616,William Shakespeare \nOR\nMiguel de Cervantes,Past,DDMonYYYY,"(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)"
3,Factual,Which from the following famous people died \n...,23-04-16,William Shakespeare \nOR\nMiguel de Cervantes,Past,DD-MM-YY,"(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)"
4,Factual,Which from the following famous people died \n...,"1616, April 23",William Shakespeare \nOR\nMiguel de Cervantes,Past,"YYYY, Mon DD","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,Numerical,What is the time 7 year and 9 month after 27Oc...,NaN,"Jul 27, 1454",Past,NaN,"(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.9647059, 0.69803923, 0.41960785)"
185,Numerical,What is the time 7 year and 9 month after 27/1...,NaN,"Jul 27, 1454",Past,NaN,"(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.5764706, 0.76862746, 0.49019608)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0.91764706, 0.6, 0.6)","(0

In [102]:
df = pd.read_csv('data_new.csv')
df

,data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Type of Q,Question,Extracted Date,Answer,Time Period,Date Format,meta-llama/Meta-Llama-3-70B-Instruct,meta-llama/Meta-Llama-3-8B-Instruct,Qwen/Qwen2.5-72B-Instruct,meta-llama/Llama-3.2-3B-Instruct,meta-llama/Llama-2-7b-chat-hf,microsoft/Phi-3.5-mini-instruct,mistralai/Mistral-7B-Instruct-v0.3,GPT-4o-mini,GPT-4o,GPT-3.5,GPT-4-turbo,CohereForAI/c4ai-command-r-plus-08-2024
1,Factual,Which from the following famous people died \n...,23041616,William Shakespeare \nOR\nMiguel de Cervantes,Past,DDMMYYYY,B) Miguel de Cervantes and D) William Shakespe...,B) Miguel de Cervantes,"B, D",C. Princess Diana,C) Princess Diana died on 23/04/1616.,Princess Diana,B) Miguel de Cervantes,D) William Shakespeare,B) Miguel de Cervantes,B) Miguel de Cervantes,D) William Shakespeare,A
2,Factual,Which from the following famous people died \n...,4231616,William Shakespeare \nOR\nMiguel de Cervantes,Past,MMDDYYYY,B) Miguel de Cervantes,"C) Princess Diana (August 31, 1997, not April ...","B, D",B) Miguel de Cervantes,C) Princess Diana,D) William Shakespeare died on 4th March 1616.,B) Miguel de Cervantes,D) William Shakespeare,B) Miguel de Cervantes,B) Miguel de Cervantes,D) William Shakespeare,B) Miguel de Cervantes.
3,Factual,Which from the following famous people died \n...,23April1616,William Shakespeare \nOR\nMiguel de Cervantes,Past,DDMonYYYY,B) Miguel de Cervantes and D) William Shakespe...,B) Miguel de Cervantes,"B, D",B) Miguel de Cervantes,"D) William Shakespeare died on April 23, 1616.",D) William Shakespeare,D) William Shakespeare,B) Miguel de Cervantes,B) Miguel de Cervantes,D) William Shakespeare,D) William Shakespeare,C) Princess Diana
4,Factual,Which from the following famous people died \n...,23-04-16,William Shakespeare \nOR\nMiguel de Cervantes,Past,DD-MM-YY,D) William Shakespeare,None of the above.,"B, D",A) Shah Jahan.,C) Princess Diana,C) Princess Diana,B) Miguel de Cervantes,D) William Shakespeare,"B, D",C) Princess Diana,D) William Shakespeare,D.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Numerical,What is the time 7 year and 9 month after 27/1...,NaN,"Jul 27, 1454",Past,NaN,The time is 3/7/1534.,"July 6, 1555",15/07/1454,"To find the date, add 7 years and 9 months to ...",The time 7 years and 9 months after 27/10/1446...,Calculating the date 7 years and 9 months afte...,"The time would be 6th July, 1555. (Using Islam...",07/07/1454,27/06/1454,09/07/1454,The date would be 27/07/1454.,"I'm sorry, the reference time you gave me is e..."
187,Numerical,What is the time 7 year and 9 month after 2710...,NaN,"Jul 27, 1454",Past,NaN,27101446 + 7 years + 9 months = 27111475,"Add 7 years and 9 months to 2710, resulting in...",27183346,I cannot provide a date in Gregorian format fo...,The time 7 years and 9 months after 27101446 i...,27113356 (27 Jan 2010 + 7 years + 9 months),The time is 27406152 hours after 27101446.\n\n...,The result is 27070753.,"July 20, 2024.",28018134,"Time after 27101446: 28091446 (adding 7 years,...",The time 7 years and 9 months after 27101446 i...
188,Numerical,What is the time 7 year and 9 month after 1027...,NaN,"Jul 27, 1454",Past,NaN,"October 27, 2040.","7 years, 9 months after 10271446 = 10311655.",10271446 + 7 years 9 months = 10353406.,I cannot help with calculations that involve a...,"Time after 10271446: 9 months = Oct 1028, 7 ye...",10360346 (calculated by adding 7 years and 9 m...,"The time would be 11 years, 9 months, and 14 s...",Add 7 years and 9 months to the date 10271446.,October 2023 plus 7 years and 9 months is July...,11039198,Time: 10429846 (Unix timestamp format).,10279141
189,Numerical,What is the time 7 year and 9 month after 10/2...,NaN,"Jul 27, 1454",Past,NaN,7 years and 9 months after 10/27/1446 is 7/26/...,"July 23, 1453.",The date is 7/27/1454.,4/3/1495

In [103]:
import pandas as pd

df = pd.read_csv('data_new.csv')
# convert the first row to header
df.columns = df.iloc[0]
df = df[1:]
df = df.reset_index(drop=True)
df_colors = pd.read_csv('colors.csv')

#drop first row
df_colors = df_colors.iloc[1:]
df_colors = df_colors.reset_index(drop=True)
#drop the last row of df
df = df.iloc[:-1]
df = df.reset_index(drop=True)
df_colors.iloc[:, :6] = df.iloc[:, :6]
COLOR_MAP = {
    (0.91764706, 0.6, 0.6): "Red",
    (0.5764706, 0.76862746, 0.49019608): "Green",
    (0.43529412, 0.65882355, 0.8627451): "Blue",
    (1.0, 1.0, 1.0): "White",  # Default background color
    (0.9647059, 0.69803923, 0.41960785): "Orange",  # Approximate orange
}

def get_color(color_tuple):
    return COLOR_MAP.get(color_tuple, "Unknown")

# List of target columns
target_columns = [
    "meta-llama/Meta-Llama-3-70B-Instruct",
    "meta-llama/Meta-Llama-3-8B-Instruct",
    "Qwen/Qwen2.5-72B-Instruct",
    "meta-llama/Llama-3.2-3B-Instruct",
    "meta-llama/Llama-2-7b-chat-hf",
    "microsoft/Phi-3.5-mini-instruct",
    "mistralai/Mistral-7B-Instruct-v0.3",
    "GPT-4o-mini",
    "GPT-4o",
    "GPT-3.5",
    "GPT-4-turbo",
    "CohereForAI/c4ai-command-r-plus-08-2024",
]

# Apply the color transformation only to the target columns
for column in target_columns:
    df_colors[column] = df_colors[column].apply(lambda x: get_color(eval(x)))

# Display the updated DataFrame
df_colors

,Type of Q,Question,Extracted Date,Answer,Time Period,Date Format,meta-llama/Meta-Llama-3-70B-Instruct,meta-llama/Meta-Llama-3-8B-Instruct,Qwen/Qwen2.5-72B-Instruct,meta-llama/Llama-3.2-3B-Instruct,meta-llama/Llama-2-7b-chat-hf,microsoft/Phi-3.5-mini-instruct,mistralai/Mistral-7B-Instruct-v0.3,GPT-4o-mini,GPT-4o,GPT-3.5,GPT-4-turbo,CohereForAI/c4ai-command-r-plus-08-2024
0,Factual,Which from the following famous people died \n...,23041616,William Shakespeare \nOR\nMiguel de Cervantes,Past,DDMMYYYY,Green,Green,Green,Red,Red,Red,Green,Green,Green,Green,Green,Red
1,Factual,Which from the following famous people died \n...,4231616,William Shakespeare \nOR\nMiguel de Cervantes,Past,MMDDYYYY,Green,Red,Green,Green,Red,Blue,Green,Green,Green,Green,Green,Green
2,Factual,Which from the following famous people died \n...,23April1616,William Shakespeare \nOR\nMiguel de Cervantes,Past,DDMonYYYY,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Red
3,Factual,Which from the following famous people died \n...,23-04-16,William Shakespeare \nOR\nMiguel de Cervantes,Past,DD-MM-YY,Green,Red,Green,Red,Red,Red,Green,Green,Green,Red,Green,Green
4,Factual,Which from the following famous people died \n...,"1616, April 23",William Shakespeare \nOR\nMiguel de Cervantes,Past,"YYYY, Mon DD",Green,Green,Green,Green,Red,Green,Green,Green,Green,Green,Green,Green
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,Numerical,What is the time 7 year and 9 month after 27Oc...,NaN,"Jul 27, 1454",Past,NaN,Red,Red,Red,Red,Red,Red,Red,Green,Green,Red,Green,Orange
185,Numerical,What is the time 7 year and 9 month after 27/1...,NaN,"Jul 27, 1454",Past,NaN,Red,Red,Red,Red,Red,Green,Red,Red,Red,Red,Green,Red
186,Numerical,What is the time 7 year and 9 month after 2710...,NaN,"Jul 27, 1454",Past,NaN,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red
187,Numerical,What is the time 7 year and 9 month after 1027...,NaN,"Jul 27, 1454",Past,NaN,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red


In [84]:
df_colors.head()

,Type of Q,Question,Extracted Date,Answer,Time Period,Date Format,meta-llama/Meta-Llama-3-70B-Instruct,meta-llama/Meta-Llama-3-8B-Instruct,Qwen/Qwen2.5-72B-Instruct,meta-llama/Llama-3.2-3B-Instruct,meta-llama/Llama-2-7b-chat-hf,microsoft/Phi-3.5-mini-instruct,mistralai/Mistral-7B-Instruct-v0.3,GPT-4o-mini,GPT-4o,GPT-3.5,GPT-4-turbo,CohereForAI/c4ai-command-r-plus-08-2024
0,Factual,Which from the following famous people died \n...,23041616,William Shakespeare \nOR\nMiguel de Cervantes,Past,DDMMYYYY,Green,Green,Green,Red,Red,Red,Green,Green,Green,Green,Green,Red
1,Factual,Which from the following famous people died \n...,4231616,William Shakespeare \nOR\nMiguel de Cervantes,Past,MMDDYYYY,Green,Red,Green,Green,Red,Blue,Green,Green,Green,Green,Green,Green
2,Factual,Which from the following famous people died \n...,23April1616,William Shakespeare \nOR\nMiguel de Cervantes,Past,DDMonYYYY,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Green,Red
3,Factual,Which from the following famous people died \n...,23-04-16,William Shakespeare \nOR\nMiguel de Cervantes,Past,DD-MM-YY,Green,Red,Green,Red,Red,Red,Green,Green,Green,Red,Green,Green
4,Factual,Which from the following famous people died \n...,"1616, April 23",William Shakespeare \nOR\nMiguel de Cervantes,Past,"YYYY, Mon DD",Green,Green,Green,Green,Red,Green,Green,Green,Green,Green,Green,Green


In [104]:
def calculate_label_percentage(df, group_by_column, labels):
    return {label: df.iloc[:, 5:].groupby(df[group_by_column]).apply(lambda x: (x == label).sum().sum() / x.size) for label in labels}

# Define the labels to calculate
labels = ["Green", "Red", "Blue", "Orange"]

# Calculate for each grouping
format_summary = calculate_label_percentage(df_colors, "Date Format", labels)
time_summary = calculate_label_percentage(df_colors, "Time Period", labels)
type_summary = calculate_label_percentage(df_colors, "Type of Q", labels)

# Combine results into DataFrames for each group
format_summary_df = pd.DataFrame(format_summary)
time_summary_df = pd.DataFrame(time_summary)
type_summary_df = pd.DataFrame(type_summary)

In [105]:
time_summary_df

,Green,Red,Blue,Orange
Time Period,,,,
Future,0.461538,0.384615,0.032967,0.043956
Past,0.407287,0.448583,0.009717,0.057490
Present,0.327249,0.445893,0.029987,0.119948


In [106]:
type_summary_df

,Green,Red,Blue,Orange
Type of Q,,,,
Common Sense,0.471933,0.407484,0.027027,0.016632
Conceptual,0.365777,0.432496,0.012559,0.112245
Factual,0.410989,0.437363,0.019780,0.054945
Numerical,0.340081,0.506073,0.048583,0.028340


In [107]:
format_summary_df

,Green,Red,Blue,Orange
Date Format,,,,
DD-MM-YY,0.369822,0.452663,0.017751,0.082840
DD/YYYY,0.461538,0.282051,0.153846,0.025641
DD/YYYY (Julian),0.291667,0.544872,0.035256,0.051282
DDMMYYYY,0.409341,0.417582,0.030220,0.065934
DDMonYYYY,0.502747,0.315934,0.002747,0.101648
MMDDYYYY,0.396450,0.437870,0.008876,0.079882
"YYYY, Mon DD",0.525641,0.285256,0.003205,0.108974
YYYY/DD,0.461538,0.384615,0.076923,0.000000
YYYY/DD (Julian),0.275641,0.573718,0.032051,0.041667
